In [1]:
from utils import *
from model_prophet import *
from tbats import TBATS
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import pearsonr
import statsmodels.api as sm
from pmdarima import auto_arima
from statsmodels.tsa.statespace import exponential_smoothing
from statsmodels.tsa.seasonal import MSTL
# from statsmodels.tsa.api import STLForeca

/Users/julius/repos/AFC/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
data = Data()

In [3]:
# TBATS model on aggregated sales data

tbats_df = pd.DataFrame({'ds': data.daily_sales.index, 'y': data.daily_sales.values})

tbats_model = TBATS(seasonal_periods=(7, 365))
tbats_fit = tbats_model.fit(tbats_df['y'])
tbats_forecast = tbats_fit.forecast(steps=28)


In [4]:
import pickle as pkl
from datetime import datetime
# Get the current time in a formatted string
current_time = datetime.now().strftime("%Y%m%d%H%M%S")

# Create a pickle file with the current time suffix
filename = f"tbats_{current_time}"
# export prohhet forecast to pickle
with open(f'forecast_pickles/{filename}.pkl', 'wb') as f:
    pkl.dump(tbats_forecast, f)

In [5]:
# Reading model parameters
print(tbats_fit.params.alpha)
print(tbats_fit.params.beta)
print(tbats_fit.params.x0)
print(tbats_fit.params.components.use_box_cox)
print(tbats_fit.params.components.seasonal_harmonics)

0.032815517598434464
None
[1402.99776323  227.02188812   32.03576335  -15.47636037  152.07750758
   83.99273378   16.43298777 -161.8464263    44.65805918   35.48924498
   44.90893971    0.            0.            0.            0.
    0.            0.        ]
False
[3 2]


In [ ]:
# Get mean distribution of sales for each product

data.merged_data['total_sales'] = data.merged_data.groupby('day')['sales'].transform('sum')
data.merged_data['percentage_sales'] = (data.merged_data['sales'] / data.merged_data['total_sales'])
mean_percentage_by_product = data.merged_data.groupby('id')['percentage_sales'].mean().reset_index()
print(mean_percentage_by_product)

In [ ]:
estimated_sales = (mean_percentage_by_product['percentage_sales'][:, np.newaxis] * tbats_forecast).round()

#print(sales_test.iloc[:, 1:])

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(data.sales_test.iloc[:, 1:], estimated_sales))

print(f'RMSE: {rmse}')